In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm

In [ ]:
import mle.tsa as mle_tsa

In [ ]:
plt.style.use('seaborn')
plt.rcParams.update({'figure.figsize': (16,8)})
sns.set_context("talk")

In [ ]:
INPATH = "data/"
INFILE1 = "EIA_Europe_Brent_Spot_Price_FOB.csv"
INFILE2 = "EIA_Cushing_OK_WTI_Spot_Price_FOB.csv"

In [ ]:
brent = pd.read_csv(INPATH + INFILE1, skiprows=4, parse_dates=['Day'], index_col='Day').sort_index()
brent.index.name = 'Date'
brent.columns = ['brent']
brent.info()

In [ ]:
brent_plot = brent['2018':'2019'].copy()

In [ ]:
brent_plot_diff = brent_plot.diff()
brent_plot_diff .name ='brent diffs'

In [ ]:
fig, axs = plt.subplots(1,3, gridspec_kw={'width_ratios': [0.35, 0.35, 0.3]}, figsize=(16,6))

ax = axs[0]
brent_plot.plot(label='Brent daily spot', ax=ax)
ax.set_ylabel('USD/bbl')

ax = axs[1]
brent_plot_diff.plot(label='Brent daily differences', ax=ax)

ax = axs[2]
sns.distplot(brent_plot_diff, vertical=True, ax=ax)
plt.show()

In [ ]:
wti = pd.read_csv(INPATH + INFILE2, skiprows=4, parse_dates=['Day'], index_col='Day').sort_index()
wti.index.name = 'Date'
wti.columns = ['wti']
wti.info()

In [ ]:
data = brent
data['wti'] = wti

In [ ]:
data['brent_diff5'] = (data['brent'] - data['brent'].shift(5)) / data['brent']
data['brent_fret5'] = (data['brent'].shift(-5) - data['brent'])/data['brent']

In [ ]:
data['brent_mave5'] = data['brent'].rolling(10).mean()
data['brent_mstd5'] = data['brent'].rolling(10).std()
data['brent_z_std'] = (data['brent']-data['brent_mave5'])/data['brent_mstd5']

In [ ]:
data['brent_mave20'] = data['brent'].rolling(20).mean()
data['brent_mave60'] = data['brent'].rolling(60).mean()
data['brent_mstd20'] = data['brent'].rolling(20).std()
data['brent_z_std'] = (data['brent']-data['brent_mave20'])/data['brent_mstd20']

data['brent_plus_1sigma'] = data['brent_mave20'] + data['brent_mstd20']
data['brent_minus_1sigma'] = data['brent_mave20'] - data['brent_mstd20']

data['brent_plus_2sigma'] = data['brent_mave20'] + 2*data['brent_mstd20']
data['brent_minus_2sigma'] = data['brent_mave20'] - 2*data['brent_mstd20']

data = data.dropna()['2013':].copy()

In [ ]:
data['2019'].plot(y=['brent', 'brent_mave20', 'brent_plus_2sigma', 'brent_minus_2sigma'])
plt.title(r'Brent 2$\sigma$ bands 2019')
plt.ylabel('(USD/bbl)')
plt.show()

In [ ]:
data['2019'].plot(y=['brent', 'brent_mave20', 'brent_mave60'])
plt.title('Brent moving average analysis in 2019')
plt.ylabel('(USD/bbl)')
plt.show()

In [ ]:
fig, axs = plt.subplots(1,2)

data['2014': '2015'].plot(y=['brent', 'brent_mave20', 'brent_mave60'], ax=axs[0])
axs[0].set_title('Brent MACD analysis in 2014-2015')

data['2020'].plot(y=['brent', 'brent_mave20', 'brent_mave60'], ax=axs[1])
axs[1].set_title('Brent MACD analysis in 2020')

for ax in axs:
    ax.set_ylabel('(USD/bbl)')
plt.show()

In [ ]:
brent_vs_wti_lr = sm.OLS(data['brent'], data['wti'])
brent_vs_wti_lr = brent_vs_wti_lr.fit()
brent_vs_wti_lr.summary()

In [ ]:
data['r_brent_vs_wti'] = brent_vs_wti_lr.resid

In [ ]:
mle_tsa.wrap_adfuller(data[['r_brent_vs_wti']], regression='c')

In [ ]:
data.loc['2018':'2019', 'r_brent_vs_wti'].describe()

In [ ]:
fig, axs = plt.subplots(2,2, sharex='col', sharey='row', gridspec_kw={'width_ratios': [0.7, 0.3]})

plt.suptitle('Brent vs WTI (2018-2019)')
data['2018':'2019'].plot(y=['brent', 'wti'], ax=axs[0,0])
sns.distplot(data.loc['2018':'2019', 'brent'], vertical=True, ax=axs[0,1])
sns.distplot(data.loc['2018':'2019', 'wti'], vertical=True, ax=axs[0,1])

data['2018':'2019'].plot(y='r_brent_vs_wti', ax=axs[1,0])
sns.distplot(data.loc['2018':'2019', 'r_brent_vs_wti'],  vertical=True, ax=axs[1,1])

for ax in axs[1,:]:
    ax.axhline(0.5, color='orange')
    ax.axhline(5, color='orange')

for ax in axs[:,0]:
    ax.set_ylabel('(USD/bbl)')
    
plt.show()

In [ ]:
data['brent_fret5'] = (data['brent'].shift(-5) - data['brent'])/data['brent']
data[['r_brent_vs_wti', 'brent_fret5']].corr()

In [ ]:
data['r_brent_vs_wti'].plot()